In [2]:
from libraries.import_export_data_objects import import_export_data as Import_Export_Data
from libraries.altair_renderings import AltairRenderings
from libraries.utility import Utility
import os
import altair as alt
my_altair = AltairRenderings()
my_data = Import_Export_Data()

In [3]:
import pandasql as psql
import pandas as pd

In [2]:
my_altair.get_time_series_gdp_chart("United States")

alt.LayerChart(...)

In [10]:
my_data.get_eu_trade_data_pcts()

,Top20group,year,Trade Group,Imports,Exports,TotalTrade,NetTrade,TotalTop20Imports,TotalTop20Exports,TotalTop20Trade,TotalTop20NetTrade,EUvWorld_Imports_tradepct,EUvWorld_Exports_tradepct,EUvWorld_TotalTrade_tradepct,EUvsWorld Imports %,EUvsWorld Exports %,EUvsWorld Total Trade %
0,EU,2014,EU,1712559.38,1777845.17,3490404.55,65285.80,5269575.03,5639336.90,10908911.86,369762.00,0.324990,0.315258,0.319959,32.50,31.53,32.00
1,EU,2014,World,3557015.65,3861491.73,7418507.31,304476.20,5269575.03,5639336.90,10908911.86,369762.00,0.675010,0.684742,0.680041,67.50,68.47,68.00
2,EU,2015,EU,1499633.84,1528817.43,3028451.27,29183.59,4456836.09,4712578.79,9169414.94,255742.74,0.336479,0.324412,0.330277,33.65,32.44,33.03
3,EU,2015,World,2957202.25,3183761.36,6140963.67,226559.15,4456836.09,4712578.79,9169414.94,255742.74,0.663521,0.675588,0.669723,66.35,67.56,66.97
4,EU,2016,EU,1519541.15,1558688.77,3078229.92,39147.62,4464489.33,4741291.80,9205780.87,276802.65,0.340362,0.328748,0.334380,34.04,32.87,33.44
5,EU,2016,World,2944948.18,3182603.03,6127550.95,237655.03,4464489.33,4741291.80,9205780.87,276802.65,0.659638,0.671252,0.665620,65.96,67.13,66.56
6,EU,2017,EU,1674052.75,1727174.26,3401227.01,53121.52,4948801.61,5217422.91,10166224.39,268621.29,0.338274,0.331040,0.334561,33.83,33.10,33.46
7,EU,2017,World,3274748.86,3490248.65,6764997.38,215499.77,4948801.61,5217422.91,10166224.39,268621.29,0.661726,0.668960,0.665439,66.17,66.90,66.54
8,EU,2018,EU,1833857.86,1893413.06,3727270.93,59555.21,5462493.61,5689531.45,11152024.96,227037.76,0.335718,0.332789,0.334224,33.57,33.28,33.42
9,EU,2018,World,3628635.75,3796118.39,7424754.03,167482.55,5462493.61,5689531.45,11152024.96,227037.76,0.664282,0.667211,0.665776,66.43,66.72,66.58


In [12]:
eu_countries=['Austria','Belgium','Croatia','Czech Republic','Denmark','Finland','France','Germany','Greece','Hungary',
'Italy','Netherlands','Poland','Portugal','Spain','Sweden']
eu_countries=pd.DataFrame(eu_countries)
eu_countries.columns=['Country']
eu_countries   

eu_data_frame=my_data.get_eu_trade_data()
        
my_sql = '''
            SELECT 
            t.*,
            g.[Imports] TotalTop20Imports,
            g.[Exports] TotalTop20Exports,
            g.[TotalTrade] TotalTop20Trade,
            g.[NetTrade] TotalTop20NetTrade,
            t.[Imports]/g.[Imports] EUvWorld_Imports_tradepct,
            t.[Exports]/g.[Exports] EUvWorld_Exports_tradepct,
            t.[TotalTrade]/g.[TotalTrade] EUvWorld_TotalTrade_tradepct
            FROM 
            (
            select
            case 
            WHEN [country] in (select distinct Country from eu_countries) then 'EU'
            WHEN [country] not in (select distinct Country from eu_countries) then 'World'
            ELSE 'World'
            END [Top20group],
            [year],
            [Trade Group],
            sum([Imports]) Imports,
            sum([Exports]) Exports,
            sum([TotalTrade]) TotalTrade,
            sum([NetTrade]) NetTrade
            from eu_data_frame
            group by [Top20group],[year],[Trade Group]
            ) t
            left join
            (
            SELECT
            [year],
            case 
            WHEN [country] in (select distinct Country from eu_countries) then 'EU'
            WHEN [country] not in (select distinct Country from eu_countries) then 'World'
            ELSE 'World'
            end [Top20group],
            sum([Imports]) Imports,
            sum([Exports]) Exports,
            sum([TotalTrade]) TotalTrade,
            sum([NetTrade]) NetTrade
            from eu_data_frame
            group by [year],[Top20group]
            ) g
            ON
            t.[Top20group]=g.[Top20group]
            and 
            t.[year]=g.[year]
        '''

my_return_data = psql.sqldf(my_sql)

my_return_data['EUvsWorld Imports %']=(my_return_data['EUvWorld_Imports_tradepct']*100).round(2)
my_return_data['EUvsWorld Exports %']=(my_return_data['EUvWorld_Exports_tradepct']*100).round(2)
my_return_data['EUvsWorld Total Trade %']=(my_return_data['EUvWorld_TotalTrade_tradepct']*100).round(2)
my_return_data

,Top20group,year,Trade Group,Imports,Exports,TotalTrade,NetTrade,TotalTop20Imports,TotalTop20Exports,TotalTop20Trade,TotalTop20NetTrade,EUvWorld_Imports_tradepct,EUvWorld_Exports_tradepct,EUvWorld_TotalTrade_tradepct,EUvsWorld Imports %,EUvsWorld Exports %,EUvsWorld Total Trade %
0,EU,2014,EU,1712559.38,1777845.17,3490404.55,65285.80,5269575.03,5639336.90,10908911.86,369762.00,0.324990,0.315258,0.319959,32.50,31.53,32.00
1,EU,2014,World,3557015.65,3861491.73,7418507.31,304476.20,5269575.03,5639336.90,10908911.86,369762.00,0.675010,0.684742,0.680041,67.50,68.47,68.00
2,EU,2015,EU,1499633.84,1528817.43,3028451.27,29183.59,4456836.09,4712578.79,9169414.94,255742.74,0.336479,0.324412,0.330277,33.65,32.44,33.03
3,EU,2015,World,2957202.25,3183761.36,6140963.67,226559.15,4456836.09,4712578.79,9169414.94,255742.74,0.663521,0.675588,0.669723,66.35,67.56,66.97
4,EU,2016,EU,1519541.15,1558688.77,3078229.92,39147.62,4464489.33,4741291.80,9205780.87,276802.65,0.340362,0.328748,0.334380,34.04,32.87,33.44
5,EU,2016,World,2944948.18,3182603.03,6127550.95,237655.03,4464489.33,4741291.80,9205780.87,276802.65,0.659638,0.671252,0.665620,65.96,67.13,66.56
6,EU,2017,EU,1674052.75,1727174.26,3401227.01,53121.52,4948801.61,5217422.91,10166224.39,268621.29,0.338274,0.331040,0.334561,33.83,33.10,33.46
7,EU,2017,World,3274748.86,3490248.65,6764997.38,215499.77,4948801.61,5217422.91,10166224.39,268621.29,0.661726,0.668960,0.665439,66.17,66.90,66.54
8,EU,2018,EU,1833857.86,1893413.06,3727270.93,59555.21,5462493.61,5689531.45,11152024.96,227037.76,0.335718,0.332789,0.334224,33.57,33.28,33.42
9,EU,2018,World,3628635.75,3796118.39,7424754.03,167482.55,5462493.61,5689531.45,11152024.96,227037.76,0.664282,0.667211,0.665776,66.43,66.72,66.58


In [31]:
my_data_frame=my_data.get_gdp_all_data()

eu_countries=['Austria','Belgium','Croatia','Czech Republic','Denmark','Finland','France','Germany','Greece','Hungary',
'Italy','Netherlands','Poland','Portugal','Spain','Sweden','United Kingdom']
eu_countries=pd.DataFrame(eu_countries)
eu_countries.columns=['Country']
eu_top_20=['France','Germany','Italy','Spain','United Kingdom','Netherlands']
eu_top_20=pd.DataFrame(eu_top_20)
eu_top_20.columns=['Country']

my_sql = '''
        select
        case 
        WHEN Country in (select distinct Country from eu_top_20) then 'Top 20 EU'
        WHEN Country in (select distinct Country from eu_countries) then 'Non Top 20 EU'
        WHEN Country not in (select distinct Country from eu_countries) then [Country]
        ELSE 'Other' END [Continental],
        [Year],
        avg([GDP Pct Growth]) [Avg GDP Pct Growth]
        from my_data_frame
        where Continental in ('China','Top 20 EU','Non Top 20 EU','United States')
        group by [Continental],[Year]
        UNION
        select
        Country as [Continental],
        [Year],
        avg([GDP Pct Growth]) [Avg GDP Pct Growth]
        from my_data_frame
        where Country='Denmark'
        group by [Continental],[Year]
        '''

my_return_data = psql.sqldf(my_sql)
my_return_data

,Continental,Year,Avg GDP Pct Growth
0,China,2000,8.49
1,China,2001,8.34
2,China,2002,9.13
3,China,2003,10.04
4,China,2004,10.11
...,...,...,...
100,United States,2016,1.71
101,United States,2017,2.33
102,United States,2018,3.00
103,United States,2019,2.16


In [27]:
eu_top_20

,Country
0,France
1,Germany
2,Italy
3,Spain
4,United Kingdom
5,Netherlands


In [4]:
gdp_growth=my_data.get_gdp_data_growth_rate('1')
gdp_growth

,Continental,Year,Avg GDP Pct Growth
0,China,2000,8.49
1,China,2001,8.34
2,China,2002,9.13
3,China,2003,10.04
4,China,2004,10.11
...,...,...,...
100,United States,2016,1.71
101,United States,2017,2.33
102,United States,2018,3.00
103,United States,2019,2.16


In [6]:
base=alt.Chart(gdp_growth)

continent=base.mark_line().encode(
    x=alt.X('Year:N'),
    y=alt.Y('Avg GDP Pct Growth:Q'),
    color='Continental:N'
    ).properties(height=350,width=500)

continent

alt.Chart(...)

In [7]:
my_data_frame=my_data.get_gdp_all_data()
my_data_frame

,Country,Metric Year,Year,AFF value added,Deposit interest rate pct,Exports G & S Pct GDP,Exports G & S Pct constant LCU,Exports G & S annual growth %,External balance on G & S pct GDP,Final Consumption Pct GDP,...,GDP Pct Growth,Imports G & S Pct GDP,Net Trade Pct GDP,Trade Pct GDP %,Trade Total,Prev Year,Year_curr,Trade Total Change,Trade Total Change %,exchange_rate
0,Argentina,2000-01-01,2000,4.679911,0.0,10.99,9.531203e+10,2.702391,-0.649694,83.116203,...,-0.79,11.64,-0.65,0.23,6.536686e+10,1999,2000,NaN,NaN,0.999500
1,Argentina,2001-01-01,2001,4.568590,0.0,11.58,9.792452e+10,2.740991,1.305761,83.068259,...,-4.41,10.27,1.31,0.22,5.911328e+10,2000,2001,-6.253578e+09,-9.57,0.999500
2,Argentina,2002-01-01,2002,10.206647,0.0,28.38,1.009533e+11,3.092955,15.012470,74.133671,...,-10.89,13.37,15.01,0.42,4.104408e+10,2001,2002,-1.806920e+10,-30.57,3.063257
3,Argentina,2003-01-01,2003,10.328167,0.0,25.93,1.070058e+11,5.995368,11.217138,74.635863,...,8.84,14.71,11.22,0.41,5.231066e+10,2002,2003,1.126658e+10,27.45,2.900629
4,Argentina,2004-01-01,2004,8.362487,0.0,23.85,1.156884e+11,8.114158,7.002593,75.446137,...,9.03,16.85,7.00,0.41,6.750975e+10,2003,2004,1.519909e+10,29.06,2.923301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,World,2016-01-01,2016,4.146136,0.0,27.38,0.000000e+00,2.670275,0.718487,0.000000,...,2.83,26.66,0.72,0.54,4.120473e+13,2015,2016,-8.582369e+11,-2.04,NaN
1172,World,2017-01-01,2017,4.110570,0.0,28.33,0.000000e+00,5.179119,0.749379,0.000000,...,3.39,27.58,0.75,0.56,4.546824e+13,2016,2017,4.263512e+12,10.35,NaN
1173,World,2018-01-01,2018,3.927336,0.0,29.21,0.000000e+00,4.353256,0.656781,0.000000,...,3.27,28.56,0.65,0.58,5.003521e+13,2017,2018,4.566965e+12,10.04,NaN
1174,World,2019-01-01,2019,4.010493,0.0,28.29,0.000000e+00,1.070526,0.535769,0.000000,...,2.60,27.76,0.53,0.56,4.903811e+13,2018,2019,-9.970979e+11,-1.99,NaN


In [14]:
eu_data_frame

,country,year,Trade Group,Imports,Exports,TotalTrade,NetTrade,Trade Partners
0,Australia,2014,EU,34501.57,8774.53,43276.12,-25727.04,16
1,Australia,2014,World,237620.27,248550.99,486171.27,10930.79,162
2,Australia,2015,EU,28492.91,7210.21,35703.12,-21282.68,16
3,Australia,2015,World,205149.12,195617.22,400766.39,-9531.87,162
4,Australia,2016,EU,30193.72,7032.54,37226.27,-23161.19,16
...,...,...,...,...,...,...,...,...
275,United States,2018,World,2561632.34,1611541.44,4173173.74,-950090.86,161
276,United States,2019,EU,376782.44,242674.09,619456.53,-134108.36,16
277,United States,2019,World,2514372.06,1570777.19,4085149.25,-943594.89,161
278,United States,2020,EU,337213.20,213784.60,550997.80,-123428.60,16


In [10]:
my_altair.get_time_series_gdp_compare_chart("United States","Japan")

alt.LayerChart(...)

In [11]:
my_altair.get_time_series_gdp_trade_trend_chart("United States")

alt.LayerChart(...)

In [12]:
my_altair.get_time_series_gdp_trade_trend_chart("Japan")

alt.LayerChart(...)

In [13]:
my_data.get_gdp_all_data()
all_gdp=my_data.get_gdp_all_data()
print(all_gdp.columns)

Index(['Country', 'Metric Year', 'Year', 'AFF value added',
       'Exports G & S Pct GDP', 'Exports G & S Pct US dollar',
       'Exports G & S Pct constant LCU', 'Exports G & S annual growth %',
       'External balance on G & S pct GDP', 'Final Consumption Pct GDP',
       'Foreign Direct Investment pct GDP', 'GDP', 'GDP (constant LCU)',
       'GDP Growth Pct', 'GDP per capita', 'GDP per capita Growth Pct',
       'GDP per capita constant LCU', 'GNI const LCU', 'GNI per capita Atlas',
       'GNI per capita Growth Pct', 'GNI per capita const LCU',
       'Govt Final Consumption expenditure Growth Pct',
       'Govt Final Consumption expenditure Pct GDP', 'Gross capital formation',
       'Gross capital formation constant LCU', 'Gross domestic savings',
       'Gross fixed capital formation',
       'Gross fixed capital formation, private sector',
       'Gross national expenditure Pct GDP',
       'Gross national expenditure constant LCU', 'Gross savings Pct GDP',
       'Gross sav

In [ ]:
def get_time_series_gdp_chart(self,source_country):

        my_data = self.my_data_object

        my_data_to_graph = my_data.get_gdp_data_by_country(source_country)

        title = "GDP per Capita " + source_country

        source_and_target_data = my_data_to_graph

        base = alt.Chart(source_and_target_data)

        line = base.mark_line().encode(
            x=alt.X('Year:N',axis=alt.Axis(title='Year')),
            y=alt.Y('GDP per capita:Q',axis=alt.Axis(title="GDP per Capita $",labelExpr='"$" + datum.value'))#,
            #color="Country:N"
            
        ).properties(
            width=700,
            height=350,
            title=title
            )

        #Throw points on so that the tool tips will work better.
        points = base.mark_circle(
            color='red',
            opacity=0.0,
            size=1000
        ).encode(
            x=alt.X('Year:N',axis=alt.Axis(title='')),
            y=alt.Y('GDP per capita:Q',axis=alt.Axis(title='')),
            tooltip=['Country','GDP $B','GDP per capita']
        ).properties(width=700)

        
        return_chart = alt.layer(line,points).configure_axis(grid=False)
        return return_chart


    def get_time_series_gdp_trade_trend_chart(self,source_country):

        my_data = self.my_data_object

        my_data_to_graph = my_data.get_gdp_data_by_country(source_country)

        title = "GDP & Trade Growth Compare " + source_country

        source_and_target_data = my_data_to_graph

        base = alt.Chart(source_and_target_data).transform_fold(['GDP Pct Growth','Trade Total Change %'])

        line = base.mark_line().encode(
            x=alt.X('Year:N',axis=alt.Axis(title='Year')),
            y=alt.Y('value:Q',axis=alt.Axis(title="GDP and Trade % Change",labelExpr='datum.value + "%"')),
            color="key:N"
        ).properties(
            width=700,
            height=350,
            title=title
            )

                #Throw points on so that the tool tips will work better.
        points = base.mark_circle(
            color='red',
            opacity=0.0,
            size=1000
        ).encode(
            x=alt.X('Year:N',axis=alt.Axis(title='')),
            y=alt.Y('value:Q',axis=alt.Axis(title='')),
            tooltip=['GDP Growth Pct','Trade Total Change %']
        ).properties(width=700)

        return_chart = alt.layer(line,points).configure_axis(grid=False)
        return return_chart

    def get_time_series_gdp_compare_chart(self,source_country,target_country):

        my_data = self.my_data_object

        my_data_to_graph = my_data.get_gdp_data_compare(source_country,target_country)

        title = "GDP Growth " + source_country + " and " + target_country + "2014 to 2020"

        source_and_target_data = my_data_to_graph

        base = alt.Chart(source_and_target_data)

        line = base.mark_line().encode(
            x=alt.X('Year:N',axis=alt.Axis(title='Year')),
            y=alt.Y('GDP Pct Growth:Q',axis=alt.Axis(title="GDP Growth %",labelExpr='datum.value + "%"')),
            color="Country:N"
            
        ).properties(
            width=700,
            height=350,
            title=title
            )

        #Throw points on so that the tool tips will work better.
        points = base.mark_circle(
            color='red',
            opacity=0.0,
            size=1000
        ).encode(
            x=alt.X('Year:N',axis=alt.Axis(title='')),
            y=alt.Y('GDP Pct Growth:Q',scale=alt.Scale(domain=(-10, 10)),axis=alt.Axis(title='')),
            tooltip=['Country','GDP Pct Growth %']
        ).properties(width=700)

        
        return_chart = alt.layer(line,points).configure_axis(grid=False)
        return return_chart

In [2]:
my_altair.get_time_series_gdp_trade_for_matrix("Japan")

alt.LayerChart(...)

In [19]:
def get_charts_for_click_from_world_map(self,source_country,width=300,height=200):
    top_5  = self.get_altaire_bar_top5_partners_for_matrix(source_country,width=width,height=height)
    trade  = self.get_import_export_balance_top_five(source_country,for_matrix=True,width=width,height=height)
    time_s = self.get_altaire_line_chart_county_trade_for_matrix(source_country,"World",width=width,height=height)
    #gdp = self.get_time_series_gdp_chart_for_matrix(source_country,width=width,height=height)
    gdp = self.get_time_series_gdp_trade_for_matrix(source_country,width=width,height=height)

    row_1 = (time_s | top_5).resolve_scale(
        color='independent')
    row_2 = (trade | gdp).resolve_scale(
        color='independent')


    my_chart = (row_1 & row_2 ).configure_axis(
    grid=False
    ).configure_view(
    strokeWidth=0
    )

In [ ]:
source_country = "United States"
target_country = "China"


#First drop down
my_chart = my_altair.get_charts_for_click_from_world_map(source_country,width=300,height=200)
my_chart

In [ ]:
from libraries.import_export_data_objects import import_export_data as Import_Export_Data
from libraries.altair_renderings import AltairRenderings
from libraries.utility import Utility
import os
import altair as alt
my_altair = AltairRenderings()
my_data = Import_Export_Data()

#First drop down
my_chart = my_altair.get_charts_for_click_from_world_map(source_country,width=300,height=200)
my_chart

#Second drop down
my_chart = my_altair.get_charts_for_country_dill_down(source_country,target_country,width=350,height=200).to_json()
my_chart

In [27]:
gdp     = my_altair.get_time_series_gdp_compare_chart_form_matrix(source_country,target_country)
gdp

alt.LayerChart(...)

In [30]:
time_s  = my_altair.get_altaire_line_chart_county_trade_for_matrix(source_country,target_country)
pie     = my_altair.get_altaire_dual_pie_chart_by_types_for_matrix(source_country,target_country, "exports")
gdp     = my_altair.get_time_series_gdp_compare_chart_form_matrix(source_country,target_country)

row_1 = (time_s | pie).resolve_scale(
                color='independent')
row_2 = (gdp ).resolve_scale(
            color='independent')


my_chart = (row_1 & row_2 ).configure_axis(
        grid=False
        ).configure_view(
        strokeWidth=0
        ).configure_view(
            stroke=None
        )

somy_chart

alt.VConcatChart(...)

In [4]:
source_country="United States"
target_country="China"

top_5  = my_altair.get_altaire_bar_top5_partners_for_matrix(source_country)
trade  = my_altair.get_import_export_balance_top_five(source_country,for_matrix=True)
time_s = my_altair.get_altaire_line_chart_county_trade_for_matrix(source_country,"World")
#gdp = my_altair.get_time_series_gdp_chart_for_matrix(source_country)#,width=width,height=height)
gdp = my_altair.get_time_series_gdp_trade_for_matrix(source_country)

row_1 = (time_s | top_5).resolve_scale(
            color='independent')
row_2 = (trade | gdp).resolve_scale(
            color='independent')

my_chart = (row_1 & row_2 ).configure_axis(
        grid=False
        ).configure_view(
        strokeWidth=0
        )

my_chart

alt.VConcatChart(...)

In [7]:
from libraries.import_export_data_objects import import_export_data as Import_Export_Data
from libraries.altair_renderings import AltairRenderings
from libraries.utility import Utility
import os
import altair as alt
my_altair = AltairRenderings()
my_data = Import_Export_Data()

source_country = "United States"
target_country = "China"


#First drop down
my_chart = my_altair.get_charts_for_click_from_world_map(source_country)
my_chart

#Second drop down
my_chart = my_altair.get_charts_for_country_dill_down(source_country,target_country,width=350,height=200)
my_chart

alt.VConcatChart(...)

In [ ]:
        eu_countries=['Austria','Belgium','Croatia','Czech Republic','Denmark','Finland','France','Germany','Greece','Hungary',
        'Italy','Netherlands','Poland','Portugal','Spain','Sweden']
        eu_countries=pd.DataFrame(eu_countries)
        eu_countries.columns=['Country']
        eu_countries

        my_sql = '''
            SELECT t.*
            FROM 
            (
            select
            [country],
            [year],
            case 
            WHEN [Trading Partner] in (select distinct Country from eu_countries) then 'EU'
            WHEN [Trading Partner]<>'European Union' then 'World'
            ELSE 'World'
            END  [Trade Group],
            sum([Imports ($M)]) Imports,
            sum([Exports ($M)]) Exports,
            sum([Total Trade ($M)]) TotalTrade,
            sum([Net Exports ($M)]) NetTrade--,
            --sum(case WHEN [Total Trade ($M)] > 0 then 1 else 0 end]) as [Trade Partners]
            from my_data_frame
            group by [country],[year],[Trade Group]
            ) t
        '''
        
        my_return_data = psql.sqldf(my_sql)

        return my_return_data